In [14]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

#### Config File

In [5]:
RAW_PATH = "../data/raw/"
PROCESSED_PATH = "../data/processed/"

#### Utils file

In [9]:
import joblib as pkl
import os

def pickle(value = None, filename = None):
    if (value and filename) is not None:
        pkl.dump(value = value, filename=filename)
    else:
        ValueError("Pickle is not possible due to missing arguments".capitalize())
        
def clean_folder(path = None):
    if path is not None:
        if os.path.exists(path):
            for file in os.listdir(path):
                os.remove(os.path.join(path, file))
            
            print("{} - path cleaned".format(path).capitalize())
        else:
            print("{} - path doesn't exist".capitalize())
    else:
        raise ValueError("Clean folder is not possible due to missing arguments".capitalize())

#### Create the DataLoader

In [13]:
import os

class Loader:
    """
    A class for loading and preprocessing the MNIST dataset.

    This class handles the downloading of the MNIST dataset, performs image transformations, and organizes the data into batches for training or testing.

    | Parameters | Description |
    |------------|-------------|
    | batch_size | int, default=128. The number of samples to include in each batch of data. |

    | Attributes | Description |
    |------------|-------------|
    | batch_size | int. The size of the batch of data. |

    | Methods    | Description |
    |------------|-------------|
    |_do_transformation() | Applies a series of transformations to the dataset images. |
    | download_mnist()    | Downloads the MNIST dataset, applies transformations, and organizes the data into batches. |

    Examples
    --------
    >>> loader = Loader(batch_size=128)
    >>> dataloader = loader.download_mnist()
    """
    def __init__(self, batch_size = 128):
        """
        Initializes the Loader with a specified batch size.

        Parameters
        ----------
        batch_size : int, optional
            The number of samples per batch. Default is 128.
        """
        self.batch_size = batch_size

    def _do_transformation(self):
        """
        Apply transformations to the dataset images.

        Returns
        -------
        torchvision.transforms.Compose
            A composed series of transformations for image processing.
        """
        transform = transforms.Compose([
            transforms.Resize((28, 28)),
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,))
        ])
        return transform

    def download_mnist(self):
        """
        Download the MNIST dataset and prepare it for training.

        Checks for dataset existence, downloads if necessary, applies transformations, and prepares a DataLoader.

        Returns
        -------
        torch.utils.data.DataLoader
            A DataLoader containing the preprocessed MNIST dataset in batches.

        Raises
        ------
        Exception
            If any errors occur during the folder cleaning or dataset processing steps.
        """
        if os.path.exists(RAW_PATH):
            try:
                clean_folder(path=RAW_PATH)
            except Exception as e:
                print("Exception caught in the section # {}".format(e))
            else:
                dataloader = datasets.MNIST(root=os.path.join(RAW_PATH), train=True, download=True, transform=self._do_transformation())
                dataloader = DataLoader(dataset=dataloader, batch_size=self.batch_size, shuffle=True)

                try:
                    if os.path.exists(PROCESSED_PATH):
                        try:
                            clean_folder(path=PROCESSED_PATH)
                        except Exception as e:
                            print("Exception caught in the section # {}".format(e))
                        else:
                            pickle(value = dataloader, filename = os.path.join(PROCESSED_PATH, "dataloader.pkl"))
                    else:
                        os.makedirs(PROCESSED_PATH)
                        print("Processed path is created in the data folder & run the code again".capitalize())

                except Exception as e:
                    print("Exception caught in the section # {}".format(e))
                else:
                    return dataloader
        else:
            os.makedirs(RAW_PATH)
            print("raw folder is created in the data folder and run again this code".capitalize())


if __name__ == "__main__":
    loader = Loader(batch_size=128)
    dataloader = loader.download_mnist()

../data/raw/ - path cleaned


100.0%


Extracting ../data/raw/MNIST/raw/train-images-idx3-ubyte.gz to ../data/raw/MNIST/raw



100.0%
27.8%

Extracting ../data/raw/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/raw/MNIST/raw



100.0%
100.0%


Extracting ../data/raw/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/raw/MNIST/raw

Extracting ../data/raw/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/raw/MNIST/raw

../data/processed/ - path cleaned


#### Discriminator

In [55]:
from collections import OrderedDict

class Discriminator(nn.Module):
    """
    A discriminator model for a Generative Adversarial Network (GAN) that classifies images as real or fake.

    This model applies a series of convolutional layers to an input image and outputs a single scalar indicating the likelihood of the image being real.

    | Parameters   | Description |
    |--------------|-------------|
    | in_channels  | int, default=1. The number of channels in the input images. |

    | Attributes   | Description |
    |--------------|-------------|
    | in_channels  | int. The number of channels in the input images. |
    | config_layer | list of tuples. Configuration for each layer in the model, specifying layer parameters. |
    | model        | nn.Sequential. The sequential model comprising the discriminator's layers. |

    | Methods      | Description |
    |--------------|-------------|
    | forward(x)   | Defines the forward pass of the discriminator. |
    | connected_layer(config_layer) | Constructs the layers of the discriminator based on the provided configuration. |

    Examples
    --------
    >>> discriminator = Discriminator(in_channels=1)
    >>> image = torch.randn(64, 1, 28, 28)
    >>> print(discriminator(image).shape)
    """
    def __init__(self, in_channels = 1):
        self.in_channels = in_channels
        super(Discriminator, self).__init__()

        self.config_layer = [
            (1, 128, 4, 2, 1, 0.2, False),
            (128, 256, 4, 2, 1, 0.2, True),
            (256, 512, 4, 2, 1, 0.2, True),
            (512, 1, 4, 2, 1),
        ]

        self.model = self.connected_layer(config_layer = self.config_layer)

    def connected_layer(self, config_layer = None):
        """
        Constructs the layers of the discriminator based on the provided configuration.

        Parameters
        ----------
        config_layer : list of tuples, optional
            The configuration for each layer in the model. If not provided, uses the instance's config_layer.

        Returns
        -------
        nn.Sequential
            A sequential container of the constructed layers.

        Raises
        ------
        Exception
            If config_layer is not provided.
        """
        layers = OrderedDict()
        if config_layer is not None:
            for index, (
                in_channels, out_channels, kernel_size, stride, padding, slope, batch_norm) in enumerate(config_layer[:-1]):
                layers[f"conv_{index+1}"] = nn.Conv2d(
                    in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=padding)
                if batch_norm:
                    layers[f"batch_norm_{index+1}"] = nn.BatchNorm2d(out_channels)
                layers[f"leaky_relu_{index+1}"] = nn.LeakyReLU(slope)

            (in_channels, out_channels, kernel_size, stride, padding) = config_layer[-1]
            layers["out_conv"] = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
            layers["out_sigmoid"] = nn.Sigmoid()

            return nn.Sequential(layers)
        else:
            raise Exception("Config layer is not passed".capitalize())

    def forward(self, x):
        if x is not None:
            x = self.model(x)
            return x.view(x.size(0), -1)
        else:
            raise Exception("Input is not passed".capitalize())


if __name__ == "__main__":
    discriminator = Discriminator(in_channels = 1)
    
    image = torch.randn(64, 1, 28, 28)
    print(discriminator(image).shape)

torch.Size([64, 1])


#### Generator

In [56]:
class Generator(nn.Module):
    def __init__(self, latent_space = 100):
        self.latent_space = latent_space
        super(Generator, self).__init__()
        
        self.layers_config = [
            (self.latent_space, 512, 4, 1, 0, True),
            (512, 256, 4, 2, 1, True),
            (256, 128, 4, 2, 1, True),
            (128, 1, 4, 2, 1)
        ]
        
        self.model = self.connected_layer(layers_config = self.layers_config)
    def connected_layer(self, layers_config = None):
        layers = OrderedDict()
        if layers_config is not None:
            for index, (in_channels, out_channels, kernel_size, stride, padding, batch_norm) in enumerate(layers_config[:-1]):
                layers["conv_transpose_{}".format(index+1)] = nn.ConvTranspose2d(in_channels = in_channels, out_channels = out_channels, kernel_size = kernel_size, stride = stride, padding = padding)
                if batch_norm:
                    layers["batch_norm_{}".format(index+1)] = nn.BatchNorm2d(num_features = out_channels)
                
                layers[f"relu_{index+1}"] = nn.ReLU(inplace = True)
                
            (in_channels, out_channels, kernel_size, stride, padding) = layers_config[-1]
            layers[f"out_conv"] = nn.ConvTranspose2d(
                in_channels = in_channels, out_channels = out_channels, kernel_size = kernel_size, stride = stride, padding = padding)
            layers[f"out_tanh"] = nn.Tanh()
            
            return nn.Sequential(layers)
                
        else:
            raise Exception("Config layer is not passed".capitalize())
    
    def forward(self, x):
        if x is not None:
            x = self.model(x)
            return x
        else:
            raise Exception("Input is not passed".capitalize())
        
        
if __name__ == "__main__":
    generator = Generator(latent_space = 100)
    noise_data = torch.randn(64, 100, 1, 1)
    print(generator(noise_data).shape)

torch.Size([64, 1, 32, 32])


#### QNet

In [72]:
class QNet(nn.Module):
    """
    A neural network model designed for Q-learning tasks, featuring convolutional layers for feature extraction and fully connected layers for action value prediction.

    The model processes input images through convolutional layers followed by fully connected layers to predict action values for each possible action, suitable for reinforcement learning environments.

    | Parameters   | Description |
    |--------------|-------------|
    | (None)       | This class does not take parameters at initialization. |

    | Attributes   | Description |
    |--------------|-------------|
    | layers_config| list of tuples. Configuration for convolutional layers specifying in_channels, out_channels, kernel_size, stride, padding, and whether to use batch normalization. |
    | model        | nn.Sequential. The sequential model comprising the convolutional layers. |
    | fc_layer     | nn.Sequential. The sequential model comprising the fully connected layers for action value prediction. |

    | Methods      | Description |
    |--------------|-------------|
    | forward(x)   | Defines the forward pass through the convolutional and fully connected layers. |
    | connected_layer(layers_config) | Constructs the convolutional layers based on the provided configuration. |
    | connected_fc_layer(in_features) | Constructs the fully connected layers for action value prediction. |

    Examples
    --------
    >>> qnet = QNet()
    >>> image = torch.randn(64, 1, 32, 32)
    >>> print(qnet(image).shape)
    """
    def __init__(self):
        super().__init__()
        
        self.layers_config = [
            (1, 128, 4, 2, 1, True),
            (128, 64, 4, 2, 1, True),
        ]
        
        self.model = self.connected_layer(layers_config = self.layers_config)
        self.fc_layer = self.connected_fc_layer(in_features = 8*8*64)
    
    def connected_layer(self, layers_config = None):
        """
        Constructs the convolutional layers of the model based on the provided configuration.

        Parameters
        ----------
        layers_config : list of tuples, optional
            Configuration for each convolutional layer. If not provided, uses the instance's layers_config.

        Returns
        -------
        nn.Sequential
            A sequential container of the constructed convolutional layers.

        Raises
        ------
        Exception
            If layers_config is not provided.
        """
        layers = OrderedDict()
        if layers_config is not None:
            for index, (in_channels, out_channels, kernel_size, stride, padding, use_norm) in enumerate(layers_config):
                layers[f"conv_{index+1}"] = nn.Conv2d(
                    in_channels = in_channels, out_channels = out_channels, kernel_size = kernel_size, stride = stride, padding = padding)
                if use_norm:
                    layers[f"batchnorm_{index+1}"] = nn.BatchNorm2d(out_channels)
                
                layers[f"relu_{index+1}"] = nn.ReLU(inplace = True)
                
            return nn.Sequential(layers)
        
        else:
            raise Exception("Config layer is not passed".capitalize())
    
    def connected_fc_layer(self, in_features = None):
        """
        Constructs the fully connected layers of the model for action value prediction.

        Parameters
        ----------
        in_features : int, optional
            The number of input features to the first fully connected layer.

        Returns
        -------
        nn.Sequential
            A sequential container of the constructed fully connected layers.

        Raises
        ------
        Exception
            If in_features is not provided.
        """
        layers = OrderedDict()
        layers[f"fc_1"] = nn.Linear(in_features = in_features, out_features = 10)
        layers[f"fc_soft"] = nn.Softmax(dim = 1)
        
        return nn.Sequential(layers)
    
    def forward(self, x):
        """
        Defines the forward pass through the model.

        Parameters
        ----------
        x : torch.Tensor
            The input tensor for the model.

        Returns
        -------
        torch.Tensor
            The output tensor after processing through the convolutional and fully connected layers.

        Raises
        ------
        Exception
            If the input tensor x is not provided.
        """
        if x is not None:
            x = self.model(x)
            x = x.view(x.size(0), -1)
            x = self.fc_layer(x)
            return x
        else:
            raise Exception("Input is not passed".capitalize())
        
        
if __name__ == "__main__":
    qnet = QNet()
    
    image = torch.randn(64, 1, 32, 32)
    print(qnet(image).shape)

torch.Size([64, 10])
